In [1]:
import import_ipynb
import sys
sys.path.append('C:\\Users\\USER\\JupyterProjects\\AdvLSTM')
from Stock_Dataset import StockDataset

importing Jupyter notebook from Stock_Dataset.ipynb


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import argparse
from Att_LSTM import attLSTM
import numpy as np
import time
from metric import metric_acc as ACC
from metric import metric_mcc as MCC
import matplotlib.pyplot as plt
import csv
import os
from loss_fn1 import adv_loss
from adv import adversarial
from Stock_datasets_csv import stock_csv_read

def train(attLSTM,adversarial ,lstm_optimizer,Partition, args): ## Data, loss function, argument
    trainloader = DataLoader(Partition['train'],
                             batch_size = args.batch_size,
                             shuffle=False, drop_last=True)
    attLSTM.train()
    adversarial.train()

    train_loss = 0.0
    for i, (x,y) in enumerate(trainloader):
        
        lstm_optimizer.zero_grad()
        
        true_y = y.squeeze().float().to(args.device)
        x = x.to(args.device)
        attLSTM.hidden = [hidden.to(args.device) for hidden in attLSTM.init_hidden()]
        es, attention_weight, attn_applied = attLSTM(x)
        # print(es.size()) [128, 20]

        y_hat, y_hat_adv = adversarial(es, true_y)
        
        loss = args.loss_fn(y_hat, y_hat_adv, true_y)
        loss.backward()


        lstm_optimizer.step()## parameter 갱신

        train_loss += loss.item()

    train_loss = train_loss / len(trainloader)
    return attLSTM, adversarial, train_loss


def validation(attLSTM,adversarial, partition, args):
    valloader = DataLoader(partition['val'],
                           batch_size=args.batch_size,
                           shuffle=False, drop_last=True)
    attLSTM.eval()
    adversarial.eval()
    
    val_loss = 0.0

    with torch.no_grad():
        for i, (x, y) in enumerate(valloader):

            true_y = y.squeeze().float().to(args.device)
            x = x.to(args.device)

            attLSTM.hidden = [attLSTM.to(args.device) for hidden in attLSTM.init_hidden()]

            es, attention_weight, attn_applied = attLSTM(x)

            y_hat, y_hat_adv = adversarial(es, true_y)

            # output_ = torch.where(output1 >= 0.5, 1.0, 0.0)
            # output_.requires_grad=True

            loss = args.loss_fn(y_hat, y_hat_adv, true_y)

            val_loss += loss.item()

        val_loss = val_loss / len(valloader)
        return attLSTM, adversarial, val_loss


def test(attLSTM,adversarial,partition, args):
    testloader = DataLoader(partition['test'],
                           batch_size=args.batch_size,
                           shuffle=False, drop_last=True)
    attLSTM.eval()

    ACC_metric = 0.0
    MCC_metric = 0.0
    with torch.no_grad():
        for i, (x, y) in enumerate(testloader):

            # feature transform
            true_y = y.squeeze().float().to(args.device)
            x = x.to(args.device)

            attLSTM.hidden = [hidden.to(args.device) for hidden in attLSTM.init_hidden()]

            es, attention_weight, attn_applied = attLSTM(x)

            y_hat, y_hat_adv = adversarial(es, true_y)

            output_ = torch.where(y_hat >= 0.0, 1.0, 0.0)

            output_.requires_grad = True

            ACC_metric += ACC(output_, true_y)
            MCC_metric += MCC(output_, true_y)

        ACC_metric = ACC_metric / len(testloader)
        MCC_metric = MCC_metric / len(testloader)

        return ACC_metric, MCC_metric



def experiment(partition, args):
    attLSTM = args.attLSTM(args.input_dim, args.hid_dim, args.output_dim, args.num_layers, args.batch_size,
                           args.dropout, args.use_bn, args.attention_head, args.attn_size,
                           activation="ReLU")
    adversarial = args.adversarial(args.batch_size, args.hid_dim, args.attn_size)

    attLSTM.to(args.device)
    adversarial.to(args.device)

    if args.optim == 'SGD':
        lstm_optimizer = optim.SGD(attLSTM.parameters(), lr=args.lr, weight_decay=args.l2)
    elif args.optim == 'RMSprop':
        lstm_optimizer = optim.RMSprop(attLSTM.parameters(), lr=args.lr, weight_decay=args.l2)
    elif args.optim == 'Adam':
        lstm_optimizer = optim.Adam(attLSTM.parameters(), lr=args.lr, weight_decay=args.l2)
    else:
        raise ValueError('In-valid optimizer choice')

    # ===== List for epoch-wise data ====== #
    train_losses = []
    val_losses = []
    # ===================================== #
    for epoch in range(args.epoch):
        ts = time.time()
        attLSTM, adversarial, train_loss = train(attLSTM, adversarial, lstm_optimizer, partition, args)

        attLSTM, adversarial, val_loss = validation(attLSTM, adversarial, partition, args)

        te = time.time()

        ## 각 에폭마다 모델을 저장하기 위한 코드
        torch.save(attLSTM.state_dict(), args.split_file_path + '\\' + str(epoch) +'_attLSTM' +'.pt')
        torch.save(adversarial.state_dict(), args.split_file_path + '\\' + str(epoch) +'_adversarial' +'.pt')

        train_losses.append(train_loss)
        val_losses.append(val_loss)

        print('Epoch {}, Loss(train/val) {:2.5f}/{:2.5f}. Took {:2.2f} sec'
              .format(epoch, train_loss, val_loss, te - ts))

    ## val_losses에서 가장 값이 최소인 위치를 저장함
    site_val_losses = val_losses.index(min(val_losses)) ## 10 epoch일 경우 0번째~9번째 까지로 나옴
    attLSTM = args.attLSTM(args.input_dim, args.hid_dim, args.output_dim, args.num_layers, args.batch_size,
                           args.dropout, args.use_bn, args.attention_head, args.attn_size,
                           activation="ReLU")
    adversarial = args.adversarial(args.batch_size, args.hid_dim, args.attn_size)

    attLSTM.to(args.device)
    adversarial.to(args.device)


    attLSTM.load_state_dict(torch.load(args.split_file_path + '\\' + str(site_val_losses) +'_attLSTM'+ '.pt'))
    adversarial.load_state_dict(torch.load(args.split_file_path + '\\' + str(site_val_losses) +'_adversarial'+ '.pt'))

    ACC, MCC = test(attLSTM, adversarial, partition, args)
    print('ACC: {}, MCC: {}'.format(ACC, MCC))

    with open(args.split_file_path + '\\'+ str(site_val_losses)+'Epoch_test_metric' +'.csv', 'w') as fd:
        print('ACC: {}, MCC: {}'.format(ACC, MCC), file=fd)

    result = {}

    result['train_losses'] = train_losses
    result['val_losses'] = val_losses
    result['ACC'] = ACC
    result['MCC'] = MCC

    return vars(args), result

importing Jupyter notebook from Att_LSTM.ipynb
importing Jupyter notebook from attention.ipynb
importing Jupyter notebook from metric.ipynb
importing Jupyter notebook from loss_fn1.ipynb
importing Jupyter notebook from adv.ipynb
importing Jupyter notebook from Stock_datasets_csv.ipynb


In [3]:

# ====== Random Seed Initialization ====== #
seed = 666
np.random.seed(seed)
torch.manual_seed(seed)

# ========= experiment setting ========== #
parser = argparse.ArgumentParser()
args = parser.parse_args("")
args.device = 'cuda' if torch.cuda.is_available() else 'cpu'

args.save_file_path = "C:\\Users\\USER\\JupyterProjects\\AdvLSTM\\results"

# ====== hyperparameter ======= #
args.batch_size = 64

args.dropout = 0.15
args.use_bn = True
args.loss_fn = adv_loss  ## loss function for classification : cross entropy
args.optim = 'Adam'
args.lr = 0.0005
args.l2 = 0.00001 #?
args.epoch = 100
# ============= model ================== #
args.attLSTM = attLSTM
args.adversarial = adversarial
# ====== att_lstm hyperparameter ======= #

args.x_frames = 10
args.y_frames = 1
args.input_dim = 10
args.hid_dim = 10
args.output_dim = 1
args.attention_head = 1
args.attn_size = 10
args.num_layers = 1
args.attLSTM_x_frames = 1


In [4]:
## 실행 파일
args.data_list = os.listdir("C:\\Users\\USER\\JupyterProjects\\AdvLSTM\\data\\kdd17\\ourpped")


with open(args.save_file_path + '\\' + 'AdvLSTM_result_t.csv', 'w', encoding='utf-8', newline='') as f:
    wr = csv.writer(f)
    wr.writerow(["model", "stock", "entire_exp_time",  "avg_test_ACC", "avg_test_ACC_std", "avg_test_MCC"])

    for data in args.data_list:
        
        stock = data.split('.')[0]

        est = time.time()
        setattr(args, 'symbol', stock)
        args.new_file_path = args.save_file_path + '\\' + "AdvLSTM_" + args.symbol
        os.makedirs(args.new_file_path)
        
        
        csv_read = stock_csv_read(data,args.x_frames,args.y_frames)
        split_data_list = csv_read.cv_split()
        
        ACC_cv = []
        for i, data in enumerate(split_data_list):
            args.split_file_path = args.new_file_path + "\\" + str(i) +"th_iter"
            os.makedirs(args.split_file_path)
            # 0번째에 index 1번째에 stock 1개가 input으로 들어감
            trainset = StockDataset(data[0])
            valset = StockDataset(data[1])
            testset = StockDataset(data[2])
        

            partition = {'train': trainset, 'val': valset, 'test': testset}


            setting, result = experiment(partition, args)
            eet = time.time()
            entire_exp_time = eet - est

            fig = plt.figure()
            plt.plot(result['train_losses'])
            plt.plot(result['val_losses'])
            plt.legend(['train_losses', 'val_losses'], fontsize=15)
            plt.xlabel('epoch', fontsize=15)
            plt.ylabel('loss', fontsize=15)
            plt.grid()
            plt.savefig(args.split_file_path + '\\' + str(args.symbol) + '_fig' + '.png')
            plt.close(fig)
            ACC_cv.append(result['ACC'])
            # csv파일에 기록하기
        ACC_cv_ar = np.array(ACC_cv)
        acc_avg = np.mean(ACC_cv_ar)
        acc_std = np.std(ACC_cv_ar)

        wr.writerow(["AdvLSTM", args.symbol, entire_exp_time, acc_avg, acc_std, result['MCC']])

Epoch 0, Loss(train/val) 70.93964/69.29726. Took 0.55 sec
Epoch 1, Loss(train/val) 70.77098/69.20193. Took 0.33 sec
Epoch 2, Loss(train/val) 70.57455/69.10251. Took 0.33 sec
Epoch 3, Loss(train/val) 70.37078/69.00876. Took 0.33 sec
Epoch 4, Loss(train/val) 70.19193/68.91489. Took 0.33 sec
Epoch 5, Loss(train/val) 70.02987/68.82022. Took 0.33 sec
Epoch 6, Loss(train/val) 69.82286/68.72392. Took 0.33 sec
Epoch 7, Loss(train/val) 69.64463/68.62774. Took 0.33 sec
Epoch 8, Loss(train/val) 69.47192/68.53207. Took 0.34 sec
Epoch 9, Loss(train/val) 69.29696/68.43465. Took 0.32 sec
Epoch 10, Loss(train/val) 69.09783/68.33858. Took 0.33 sec
Epoch 11, Loss(train/val) 68.90454/68.25152. Took 0.33 sec
Epoch 12, Loss(train/val) 68.77487/68.17127. Took 0.33 sec
Epoch 13, Loss(train/val) 68.51983/68.10105. Took 0.33 sec
Epoch 14, Loss(train/val) 68.48486/68.03644. Took 0.32 sec
Epoch 15, Loss(train/val) 68.35212/67.97826. Took 0.34 sec
Epoch 16, Loss(train/val) 68.20467/67.92469. Took 0.33 sec
Epoch 1